# < 직장인구_상권별_전처리 >
# < 4조 : 이가희 >

In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns

# plt.rc('font', family='AppleGothic')
plt.rc('axes', unicode_minus=False)


# 데이터 및 분석 설명

**상권**별 **직장인구**에 관한 데이터 입니다   

**상권**별이기에 행정동 코드가 따로없고 대신 상권별로 구분되어있습니다(골목상권/발달상권/전통시장/관광특구)  

따라서 다른 상권구분이 되어있는데이터와 엮어서 분석하면 될 것 같습니다  

상권별로 전처리를 진행하였고, 2023년 1~3분기 데이터만 가지고 진행하였습니다  

**기준년도 / 상권 / 총 직장 인구수 / 성별 / 연령대별** 로 나뉘어져 있습니다   

성별 + 연령이 묶여있는데이터까지 모두 처리시 데이터의 양이 너무 작아져서 (약 96개 남음) 제외하고 처리했습니다

10대 인구는 연령 특성상 직장인의 수가 작아 매우 적은 값을 보이는 경향이 있습니다  

전처리 후 306 -> 213개의 데이터가 되었습니다  

> 함수 
- iqr_cal : iqr 을 계산해주는 함수 
- find_outlier_value : 이창치 탐지 함수
- find_outlier_value_len : 이상치 개수 탐지
- (( find_outlier -> find_outliers)) : 이상치 탐지 
- remove_outliers : 이상치를 데이터 프레임에서 제거 

> 데이터프레임 
- 직장인구_상권 / 직장인구_상권_raw
- A / A_raw : 골목상권
- D / D_raw : 발달상권
- R / R_raw : 전통시장
- U / U_raw : 관광특구

> 변수정리    
label : 박스플롯을 만들때 라벨을 임시로 넣어놓은 공간

# 진행과정 
0. 결측치 확인
1. 2023년 1~3분기 데이터 추출
2. 상권별로 데이터 프레임 나눔
3. 전처리를 위한 함수 작성
4. 상권별 이상치 개수 확인 / 제거 
5. 박스플롯으로 이상치 전후 데이터 시각화 
6. 데이터 통합
7. 전처리 전후의 통계값 비교
8. 데이터 프레임을 csv 파일로 저장 

# 앞으로 진행사항 
1. 상권코드 별로 분류..?
2. 상권별로 묶어진 데이터와 함께 상권코드를 결합해서(점포-상권) 분석을 해보면 좋을것 같습니다  
2-1. 상권별로 성별/연령대 별 어떠한 서비스 업종(음식점) 이 인기가 많은지   
2-2. 직장인구-상권시각화에서 예시로 남성 30대의 인구가 많다 -> 어떠한 서비스업종이 많고 매출이 많은지 그것이 유동인구와 관계가 있는지 확인
3. 인구의 이상치(큰 인구수)의 값 만따로봐서 분석하기 

# 고민사항
1. 상권코드를 어떻게 구분할지 고민이다. 하나하나 처리하기에는 각자 변수가 너무 달라서 처리도 어려울것 같고 기준을 어떻게 잡아야할지 또 그 기준이 주관적인것 같아 고민중 
2. 성별+연령대별 데이터까지 모두 전처리를 하면 데이터가 96개이고 안하면 213개? 이정도 되는데 성별+연령대 데이터가 필요할까요? 필요할지 안할지 여부에 따라서 다시 진행하겠습니다

# 유의사항
1. 제 맥북이라... 경로가 절대경로 + 인코딩 값이 조금 다릅니다... 바꿔서 사용하셔야 해요 
2. 폰트도 애플고딕에서 나눔고딕으로 바꿔서 사용하셔야 플롯이 보일것 같아요... 

In [ ]:
# 직장인구_상권= pd.read_csv('/Users/leegahee/git_folder/semi_project/강남구/직장인구-상권.csv',encoding='EUC-KR')
# 직장인구_상권_raw = 직장인구_상권.copy()
# 직장인구_상권.head()
# 맥용 

In [ ]:
# 다른분들은 이거 돌려주세요 !
path= '../csv/직장인구-상권.csv'
직장인구_상권_raw = pd.read_csv(path, encoding='cp949')
직장인구_상권 = 직장인구_상권_raw.copy()
plt.rcParams['font.family'] = 'NanumGothic'

# 0. 결측치 확인

In [ ]:
직장인구_상권.isna().sum()

# 1. 2023년 1~3분기 데이터 추출

In [ ]:
직장인구_상권['기준_년분기_코드'].unique()

In [ ]:
condition = 직장인구_상권['기준_년분기_코드'].isin([20231, 20232, 20233, 20234])
직장인구_상권 = 직장인구_상권[condition]
직장인구_상권_raw = 직장인구_상권
직장인구_상권['기준_년분기_코드'].unique()

In [ ]:
직장인구_상권.info()

In [ ]:
직장인구_상권['상권_구분_코드'].unique()
# A : 골목상권
# D : 발달상권
# R : 전통시장
# U : 관광특구

In [ ]:
A_raw = 직장인구_상권[직장인구_상권['상권_구분_코드'] == 'A']
D_raw = 직장인구_상권[직장인구_상권['상권_구분_코드'] == 'D']
R_raw = 직장인구_상권[직장인구_상권['상권_구분_코드'] == 'R']
U_raw = 직장인구_상권[직장인구_상권['상권_구분_코드'] == 'U']
A = A_raw.copy()
D = D_raw.copy()
R = R_raw.copy()
U = U_raw.copy()


In [ ]:
len(A),len(D),len(R),len(U)

# 3. 전처리를 위한 함수 작성

IQR 을 이용하여 이상치를 탐지하였습니다

In [ ]:
import matplotlib.pyplot as plt

# IQR 계산
def iqr_cal(df, column):

    q25 = np.percentile(df[column].values,25)
    q75 = np.percentile(df[column].values,75)
    iqr = q75-q25
    iqr_weight = iqr*1.5
    low = q25 - iqr_weight
    high = q75+iqr_weight
    return low,high


In [ ]:
# 이상치 탐지
def find_outlier_value(df,column):
    temp = []
    for i in df[column] :
        if  (i < iqr_cal(df,column)[0]) or (i > iqr_cal(df,column)[1]) :
            temp.append(i)
    return temp


In [ ]:
# 이상치 탐지
def find_outlier_value_len(df,column):
    temp = []
    for i in df[column] :
        if  (i < iqr_cal(df,column)[0]) or (i > iqr_cal(df,column)[1]) :
            temp.append(i)
    print(f'{column} 에 이상치 {len(temp)} 개 존재')
    return temp


In [ ]:
def find_outliers(df):
    outliers = []
    for i in range(5, 14):
        outliers.append(find_outlier_value_len(df, df.columns[i]))
    return outliers


In [ ]:
def remove_outliers(df):
    for i in range(5,14):
        outliers = find_outlier_value(df,df.columns[i])
        df = df[~df.iloc[:,i].isin(outliers)]
    return df

# 4. 상권별 이상치 개수 확인 / 제거  
# 5. 박스플롯으로 이상치 전후 데이터 시각화 

# A : 골목상권

In [ ]:
find_outliers(A)

In [ ]:
A = remove_outliers(A)

In [ ]:
ax, pig = plt.subplots(figsize=(12,5))
label = ['총','남성','여성']
plt.title('A 가공전 (총/남/여) ')
plt.boxplot([A_raw.iloc[:,5], A_raw.iloc[:,6], A_raw.iloc[:,7]],labels=label)
plt.show()

In [ ]:
ax, pig = plt.subplots(figsize=(12,5))
label = ['총','남성','여성']
plt.title('A 가공후 (총/남/여) ')
plt.boxplot([A.iloc[:,5], A.iloc[:,6], A.iloc[:,7]],labels=label)
plt.show()

In [ ]:
ax, pig = plt.subplots(figsize=(12,5))
label = ['10대','20대','30대','40대','50대','60대이상']
plt.title('A 가공전 (연령대)')
plt.boxplot([A_raw.iloc[:,8], A_raw.iloc[:,9], A_raw.iloc[:,10],A_raw.iloc[:,11],A_raw.iloc[:,12],A_raw.iloc[:,13]],labels=label)
plt.show()

In [ ]:
ax, pig = plt.subplots(figsize=(12,5))
label = ['10대','20대','30대','40대','50대','60대이상']
plt.title('A 가공후 (연령대)')
plt.boxplot([A.iloc[:,8], A.iloc[:,9], A.iloc[:,10],A.iloc[:,11],A.iloc[:,12],A.iloc[:,13]],labels=label)
plt.show()

In [ ]:
ax, pig = plt.subplots(figsize=(12,5))
label = ['10대 남성','20대 남성','30대 남성','40대 남성','50대 남성','60대이상 남성']
plt.title('A 가공전 (남성+연령대)')
plt.boxplot([A_raw.iloc[:,14], A_raw.iloc[:,15], A_raw.iloc[:,16],A_raw.iloc[:,17],A_raw.iloc[:,18],A_raw.iloc[:,19]],labels=label)
plt.show()

In [ ]:
ax, pig = plt.subplots(figsize=(12,5))
label = ['10대 남성','20대 남성','30대 남성','40대 남성','50대 남성','60대이상 남성']
plt.title('A 가공후 (남성+연령대)')
plt.boxplot([A.iloc[:,14], A.iloc[:,15], A.iloc[:,16],A.iloc[:,17],A.iloc[:,18],A.iloc[:,19]],labels=label)
plt.show()

In [ ]:
ax, pig = plt.subplots(figsize=(12,5))
label = ['10대 여성','20대 여성','30대 여성','40대 여성','50대 여성','60대이상 여성']
plt.title('A 가공전 (여성+연령대)')
plt.boxplot([A_raw.iloc[:,20], A_raw.iloc[:,21], A_raw.iloc[:,22],A_raw.iloc[:,23],A_raw.iloc[:,24],A_raw.iloc[:,25]],labels=label)
plt.show()

In [ ]:
ax, pig = plt.subplots(figsize=(12,5))
label = ['10대 여성','20대 여성','30대 여성','40대 여성','50대 여성','60대이상 여성']
plt.title('A 가공후 (여성+연령대)')
plt.boxplot([A.iloc[:,20], A.iloc[:,21], A.iloc[:,22],A.iloc[:,23],A.iloc[:,24],A.iloc[:,25]],labels=label)
plt.show()

# D : 발달상권

In [ ]:
find_outliers(D)

In [ ]:
D = remove_outliers(D)

In [ ]:
ax, pig = plt.subplots(figsize=(12,5))
label = ['총','남성','여성']
plt.title('D 가공전')
plt.boxplot([D_raw.iloc[:,5], D_raw.iloc[:,6], D_raw.iloc[:,7]],labels=label)
plt.show()

In [ ]:
ax, pig = plt.subplots(figsize=(12,5))
label = ['총','남성','여성']
plt.title('D 가공후')
plt.boxplot([D.iloc[:,5], D.iloc[:,6], D.iloc[:,7]],labels=label)
plt.show()

In [ ]:
ax, pig = plt.subplots(figsize=(12,5))
label = ['10대','20대','30대','40대','50대','60대이상']
plt.title('D 가공전')
plt.boxplot([D_raw.iloc[:,8], D_raw.iloc[:,9], D_raw.iloc[:,10],D_raw.iloc[:,11],D_raw.iloc[:,12],D_raw.iloc[:,13]],labels=label)
plt.show()

In [ ]:
ax, pig = plt.subplots(figsize=(12,5))
label = ['10대','20대','30대','40대','50대','60대이상']
plt.title('D 가공후')
plt.boxplot([D.iloc[:,8], D.iloc[:,9], D.iloc[:,10],D.iloc[:,11],D.iloc[:,12],D.iloc[:,13]],labels=label)
plt.show()

# R : 전통시장

In [ ]:
find_outliers(R)

In [ ]:
R = remove_outliers(R)

In [ ]:
ax, pig = plt.subplots(figsize=(12,5))
label = ['총','남성','여성']
plt.title('R 가공전')
plt.boxplot([R_raw.iloc[:,5], R_raw.iloc[:,6], R_raw.iloc[:,7]],labels=label)
plt.show()

In [ ]:
ax, pig = plt.subplots(figsize=(12,5))
label = ['총','남성','여성']
plt.title('R 가공후')
plt.boxplot([R.iloc[:,5], R.iloc[:,6], R.iloc[:,7]],labels=label)
plt.show()

In [ ]:
ax, pig = plt.subplots(figsize=(12,5))
label = ['10대','20대','30대','40대','50대','60대이상']
plt.title('R 가공전')
plt.boxplot([R_raw.iloc[:,8], R_raw.iloc[:,9], R_raw.iloc[:,10],R_raw.iloc[:,11],R_raw.iloc[:,12],R_raw.iloc[:,13]],labels=label)
plt.show()

In [ ]:
ax, pig = plt.subplots(figsize=(12,5))
label = ['10대','20대','30대','40대','50대','60대이상']
plt.title('R 가공후')
plt.boxplot([R.iloc[:,8], R.iloc[:,9], R.iloc[:,10],R.iloc[:,11],R.iloc[:,12],R.iloc[:,13]],labels=label)
plt.show()

# U : 관광특구

In [ ]:
find_outliers(U)

# 6. 데이터 통합

In [ ]:
직장인구_상권 = pd.concat([A,D,R,U])
직장인구_상권['상권_구분_코드'].unique()

In [ ]:
직장인구_상권.info()

In [ ]:
직장인구_상권.to_csv('./직장인구상권_전처리.csv')

남성/여성 연령대는 분석이 너무 복잡해지기도 하고 필요하지도 않을것 같아 제외

In [ ]:
직장인구_상권 = 직장인구_상권.iloc[:,:14]
직장인구_상권.head()

# 7. 전처리 전후의 통계값 비교

In [ ]:
직장인구_상권_raw.iloc[:,5:14].describe()

In [ ]:
직장인구_상권.iloc[:,5:14].describe()

# 8. 데이터 프레임을 csv 파일로 저장 

In [ ]:
직장인구_상권.head()

In [ ]:
직장인구_상권.to_csv('./직장인구상권_전처리_전체.csv')

In [ ]:
직장인구_상권 = 직장인구_상권.loc[ :,['기준_년분기_코드','상권_구분_코드_명','상권_코드','상권_코드_명','총_직장_인구_수','남성_직장_인구_수','여성_직장_인구_수','연령대_10_직장_인구_수','연령대_20_직장_인구_수','연령대_30_직장_인구_수','연령대_40_직장_인구_수','연령대_50_직장_인구_수','연령대_60_이상_직장_인구_수'] ]

In [ ]:
직장인구_상권.to_csv('./직장인구상권_전처리_요약.csv')